# Lab 02

## The dataset

### 1. How many splits does the dataset has ?

In [45]:
from datasets import get_dataset_split_names

get_dataset_split_names("imdb")

['train', 'test', 'unsupervised']

There is three splits: train, test and unsupervised.

### 2. How big are these splits ?

In [46]:
from datasets import load_dataset

dataset = load_dataset("imdb")
dataset

Found cached dataset imdb (C:/Users/carok/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)
100%|██████████| 3/3 [00:00<00:00, 230.79it/s]


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [47]:
dataset.num_rows

{'train': 25000, 'test': 25000, 'unsupervised': 50000}

### 3. What is the proportion of each class on the supervised splits ?

In [48]:
dataset.num_columns

{'train': 2, 'test': 2, 'unsupervised': 2}

In [49]:
dataset.column_names

{'train': ['text', 'label'],
 'test': ['text', 'label'],
 'unsupervised': ['text', 'label']}

In [50]:
dataset.data['train']['label'].value_counts()

-- is_valid: all not null
-- child 0 type: int64
  [
    0,
    1
  ]
-- child 1 type: int64
  [
    12500,
    12500
  ]

In [51]:
dataset.data['test']['label'].value_counts()

-- is_valid: all not null
-- child 0 type: int64
  [
    0,
    1
  ]
-- child 1 type: int64
  [
    12500,
    12500
  ]

## Naive Bayes classifier

In [55]:
import string

In [61]:
def lower_and_remove_punctuation(x):
    x['text'] = x['text'].lower().translate(str.maketrans('', '', string.punctuation))
    return x

In [ ]:
d = dataset.map(lower_and_remove_punctuation)
d.data['train']['text'][0]

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

In [14]:
corpus = dataset.data['train']['text'].to_numpy()[:100]
X = vectorizer.fit_transform(corpus)

In [15]:
vectorizer.get_feature_names_out()

array(['000', '10', '100', ..., 'zone', 'zoom', 'zooms'], dtype=object)

In [16]:
X.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)